In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install --upgrade tensorflow
!pip install --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 28.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.

In [3]:
!cp /content/gdrive/MyDrive/kaggle_dataset/garbage-classification.zip /content

In [4]:
!unzip /content/gdrive/MyDrive/kaggle_dataset/garbage-classification.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/Garbage classification/Garbage classification/cardboard/cardboard152.jpg  
  inflating: /content/Garbage classification/Garbage classification/cardboard/cardboard153.jpg  
  inflating: /content/Garbage classification/Garbage classification/cardboard/cardboard154.jpg  
  inflating: /content/Garbage classification/Garbage classification/cardboard/cardboard155.jpg  
  inflating: /content/Garbage classification/Garbage classification/cardboard/cardboard156.jpg  
  inflating: /content/Garbage classification/Garbage classification/cardboard/cardboard157.jpg  
  inflating: /content/Garbage classification/Garbage classification/cardboard/cardboard158.jpg  
  inflating: /content/Garbage classification/Garbage classification/cardboard/cardboard159.jpg  
  inflating: /content/Garbage classification/Garbage classification/cardboard/cardboard16.jpg  
  inflating: /content/Garbage classification/Garbage classification/cardboard

In [5]:
import sys
sys.path.append('/content/garbage classification/Garbage classification')

In [6]:
!cd /content/garbage\ classification/Garbage\ classification

In [7]:
from numpy.core.multiarray import fromiter
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense
from keras.models import Sequential

import glob, os, random

In [8]:
import tensorflow as tf


In [9]:
import pandas as pd


In [10]:
base_path = '../content/garbage classification/Garbage classification'

img_list = glob.glob(os.path.join(base_path, '*/*.jpg'))

print(len(img_list))

2527


In [11]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.1
)

In [12]:

test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

In [13]:
train_generator = train_datagen.flow_from_directory(
    base_path,
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical',
    subset='training',
    seed=0
)

Found 2276 images belonging to 6 classes.


In [14]:
validation_generator = test_datagen.flow_from_directory(
    base_path,
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical',
    subset='validation',
    seed=0
)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

print(labels)

Found 251 images belonging to 6 classes.
{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}


In [15]:
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [18]:
from tensorflow.keras.models import Sequential

In [19]:
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())

In [20]:
model.add(Dense(kernel_initializer='uniform',activation='relu',units=150))
model.add(Dense(kernel_initializer='uniform',activation='relu',units=68))
model.add(Dense(kernel_initializer='uniform',activation='softmax',units=6))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 63, 63, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 31, 31, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 29, 29, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 32)        0

In [22]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [24]:
res=model.fit_generator(train_generator, epochs=3, validation_data=validation_generator)


<ipython-input-24-859546833575>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  res=model.fit_generator(train_generator, epochs=3, validation_data=validation_generator)


Epoch 1/3
36/36 [==============================] - 92s 3s/step - loss: 1.7028 - accuracy: 0.2320 - val_loss: 1.6513 - val_accuracy: 0.2430
Epoch 2/3
36/36 [==============================] - 88s 2s/step - loss: 1.6268 - accuracy: 0.3093 - val_loss: 1.5673 - val_accuracy: 0.2869
Epoch 3/3
36/36 [==============================] - 87s 2s/step - loss: 1.4928 - accuracy: 0.3730 - val_loss: 1.6205 - val_accuracy: 0.3147


In [25]:
model.save('Garbage1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model=load_model('Garbage1.h5')

In [27]:
img=image.load_img('/content/Garbage classification/Garbage classification/plastic/plastic102.jpg',
                   target_size=(128,128)
                   )
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)

In [28]:
a=np.argmax(model.predict(x), axis=1)

1/1 [==============================] - 0s 267ms/step


In [29]:
index=['0','1','2','3','4','5']
result =str(index[a[0]])
result

'4'

In [30]:
train_generator.class_indices

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}

In [31]:
index1=['cardboard','glass','metal','paper','plastic','trash']
result1=str(index[a[0]])
result1

'4'

In [32]:
!pip install nbconvert